In [7]:
import requests
import json
from langdetect import detect


api_key = 'AIzaSyD2227tpA'

# 获取包含"ChatGPT"关键字的最受欢迎的100个视频
search_url = 'https://www.googleapis.com/youtube/v3/search'
search_params = {
    'part': 'snippet',
    'q': 'ChatGPT',
    'type': 'video',
    'order': 'viewCount',
    'maxResults': 50,
    'relevanceLanguage': 'en',
    'key': api_key,
}

response = requests.get(search_url, params=search_params)
videos = response.json()['items']

# Initialize a dictionary to hold the video IDs and their comments
video_comments = {}

# 获取每个视频的前十个评论
comments_url = 'https://www.googleapis.com/youtube/v3/commentThreads'
for video in videos:
    video_id = video['id']['videoId']

    comments_params = {
        'part': 'snippet',
        'videoId': video_id,
        'order': 'relevance',
        'maxResults': 100,  # Increase the max results to filter them later
        'textFormat': 'plainText',
        'key': api_key,
    }

    response = requests.get(comments_url, params=comments_params)
    comments = response.json()['items']

    english_comments = []
    for comment in comments:
        text = comment['snippet']['topLevelComment']['snippet']['textDisplay']
        try:
            # Detect the language of the comment
            if detect(text) == 'en':
                english_comments.append(text)
        except:
            pass

        # Stop if we have 10 english comments
        if len(english_comments) == 10:
            break

    # Save the comments for this video in the dictionary
    video_comments[video_id] = english_comments

# Convert the dictionary to a JSON string
#json_data = json.dumps(video_comments)

# Print the JSON string
#print(json_data)

# Save the dictionary to a JSON file
with open('video_comments.json', 'w') as f:
    json.dump(video_comments, f)